In [1]:
## 找出今天數量，各種均量
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
import _beowFmt as fm 

def combineStr(r):
    stockId, ma1, ma5, ma20, ma60, ma120, ma240 = r["stockId"], r["ma1"], r["ma5"], r["ma20"], r["ma60"], r["ma120"], r["ma240"]
    return f"{stockId},{ma1},{ma5},{ma20},{ma60},{ma120},{ma240}"

##
volumns  = data.get("price:成交股數").fillna(0).astype(float)[250: ]  
v = (volumns/1000).round(0).astype(int)
vMa5 = v.rolling(5, min_periods=1).mean().round(0).astype(int).tail(1)
vMa20 = v.rolling(20, min_periods=1).mean().round(0).astype(int).tail(1)
vMa60 = v.rolling(60, min_periods=1).mean().round(0).astype(int).tail(1)
vMa120 = v.rolling(120, min_periods=1).mean().round(0).astype(int).tail(1)
vMa240 = v.rolling(240, min_periods=1).mean().round(0).astype(int).tail(1)

## 本益比、EPS
pER = data.get('price_earning_ratio:本益比')
print(pER["2328"])

sss = data.get('trading_attention')
# print(sss["2328"])

vdf = v.iloc[-1:].transpose()
vdf["stock"] = vdf.index
vdf.columns = ["ma1", "stockId"]
vdf["ma5"] = vMa5.iloc[-1:].transpose()
vdf["ma20"] = vMa20.iloc[-1:].transpose()
vdf["ma60"] = vMa60.iloc[-1:].transpose()
vdf["ma120"] = vMa120.iloc[-1:].transpose()
vdf["ma240"] = vMa240.iloc[-1:].transpose()
vdf["comp"] = vdf.apply(combineStr, axis = 1)
vdf2 = vdf[ (vdf["stockId"].str.len() == 4)].dropna(axis = 0, how ='any')

# print(vdf)
s1 = 'id,yVolume,ma5,ma20,ma60,ma120,ma240\n'
for c in vdf2["comp"].tolist():
    s1 += f"{c}\n"
# print(s1)
fm.write_LogFile(f"volumeData/ma_20230307.csv", s1)

輸入成功!


date
2010-01-04    52.02
2010-01-05    51.19
2010-01-06    52.20
2010-01-07    50.46
2010-01-08    51.10
              ...  
2023-03-01    16.34
2023-03-02    16.10
2023-03-03    16.14
2023-03-06    16.38
2023-03-07    16.43
Name: 2328, Length: 3236, dtype: float64


In [2]:
#前置作業準備，只要一次!!
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

df5k = pd.read_json("paras/mapping5K.json")
dfv = pd.read_csv("volumeData/ma_20230306.csv")
dfv["id"] = dfv["id"].astype("string")
# dfv = dfv.drop(columns=['close'])
print(dfv[dfv["id"] == "1305"])

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

url4 = "https://www.wantgoo.com/investrue/all-alive"
r3 = requests.get(url4, headers = headers).content
soup = BeautifulSoup(r3, "html.parser")
rr3 = soup.prettify()
dfn = pd.read_json(rr3)
dfn = dfn[(dfn["id"]>="1101") & (dfn["id"]<="9999") & (dfn["id"].str.len() == 4)]
dfn = dfn.drop(columns=['type','country','url', 'industries'])
dfn["id"] = dfn["id"].astype("string")

### 計算量比參數 ###
import time
def getRate5K():
    localtime = time.localtime()
    nowTime = time.strftime("%H:%M", localtime)
    Rate5k = df5k[(df5k["e"] >= nowTime) & (df5k["b"] <= nowTime)]["w"].values[0]  #五分K量比預估放大參數 
    print(f"目前時間：{nowTime},預估量放大係數:{Rate5k}")
    return Rate5k

print(getRate5K())

      id  yVolume   ma5  ma20  ma60  ma120  ma240
52  1305     7943  4022  2864  5083   3105   2382
目前時間：09:35,預估量放大係數:4.0
4.0


## 本益比成長

In [2]:
#----取得股票名稱----
basic2 = data.get('company_basic_info')
dfff = basic2[["stock_id","公司簡稱","產業類別"]]
df_b = dfff.set_index('stock_id')
df_b

def getStockName(stockId):
    return df_b.loc[stockId]["公司簡稱"]
#-------------------------------------------------------------------

import finlab
finlab.login(open("config.txt", "r").read())
from finlab import data
from finlab.backtest import sim

pe            = data.get('price_earning_ratio:本益比')
rev           = data.get('monthly_revenue:當月營收')
rev_ma3       = rev.average(2) #3
rev_ma12      = rev.average(12) #12
營業利益成長率 = data.get('fundamental_features:營業利益成長率')
peg           = (pe/營業利益成長率)
cond1 = rev_ma3/rev_ma12 > 1.1
cond2 = rev/rev.shift() > 0.9

stocks = {"1777","1795","2482","2495"}

# pe2 = pe[stocks][-1:]
# peg2 = peg[stocks][-1:]
# rev2 = rev[stocks][-3:]
# print(pe2)
# print(peg2*100)
# print(rev2)


cond_all  = cond1 & cond2
result    = peg*(cond_all)
position  = result[result>0].is_smallest(90).reindex(rev.index_str_to_date().index, method='ffill')

# position = cond1m & cond1q & cond1y & cond2m & cond2q & cond2yy & cond3y & cond3q & cond4
# print(position)

dff = position[-1:].transpose()
dff["stock"] = dff.index
dff.columns = ["val", "stock_id"]
df3 = dff[ (dff["val"] == True) & (dff["stock_id"].str.len() == 4)].dropna()

print(f"總共 {len(df3)} 檔股票")
print(df3)


import _beowFmt as fm 
s1, s2, today = '', '', position.iloc[-1:].index[0].strftime("%Y%m%d") 

for c in df3["stock_id"].tolist():
    s1 += f"{c}.TW,"
    s2 += f"{c}-{getStockName(c)}\n"
fm.write_LogFile(f"datas-xq/{today}_本益比成長.csv", s1)
fm.write_LogFile(f"datas/{today}_本益比成長.csv", s2)

輸入成功!


總共 9 檔股票
       val stock_id
2809  True     2809
3004  True     3004
3029  True     3029
3046  True     3046
4763  True     4763
5284  True     5284
5312  True     5312
6016  True     6016
6235  True     6235


In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 
from IPython.display import display

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180)                       # 设置打印宽度(**重要**)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

r = requests.get("https://www.wantgoo.com/investrue/all-quote-info", headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
df = pd.read_json(rr)
df = df.drop(columns=['tradeDate', 'time','millionAmount'])
df["id"] = df["id"].astype("string")
df = pd.merge(df, dfn, left_on="id", right_on="id")  ## 結合股票名稱
df["float"] = df["close"].astype("float")
df["amp"] = ((df["close"] - df["previousClose"])/df["previousClose"] * 100).round(2)
df["jump"] = df["open"] - df["previousClose"]
df["jumpRate"] = (df["jump"] / df["open"] * 100).round(2)
df = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["id"].str.len() == 4) & (df["close"] > df["previousClose"])].sort_values("jumpRate", ascending=False)
# df.info()
# print(df.head(50))

r2 = requests.get("https://www.wantgoo.com/stock/all-turnover-rates", headers = headers).content
soup = BeautifulSoup(r2, "html.parser")
rr2 = soup.prettify()
dfa = pd.read_json(rr2)
# print(dfv)

dfa["investrueId"] = dfa["investrueId"].astype("string")
dfb = pd.merge(df, dfa, left_on="id", right_on="investrueId")
dfb = dfb.drop(columns=['investrueId'])
dfb = dfb[(dfb["value"] > 1)]             #周轉率
dfb["周轉率"] = dfb["value"].round(2)


dfc = pd.merge(dfb, dfv, left_on="id", right_on="id")
dfc["預估量"] = (dfc["volume"] * getRate5K()).astype(int)        ##預估量
dfc["量比"] = (dfc["預估量"] / dfc["yVolume"]).round(2)
dfc["週量比"] = (dfc["預估量"] / dfc["ma5"]).round(2)
dfc["月量比"] = (dfc["預估量"] / dfc["ma20"]).round(2)
dfc["季量比"] = (dfc["預估量"] / dfc["ma60"]).round(2)
dfc["半年量比"] = (dfc["預估量"] / dfc["ma120"]).round(2)
dfc["年量比"] = (dfc["預估量"] / dfc["ma240"]).round(2)
dfc["量比周轉"] = ((dfc["量比"] + dfc["value"]) / 2).round(4)
dfc2 = dfc[(dfc["量比"] > 1) & (dfc["yVolume"] > 2000) & (dfc["close"] < 200) & (dfc["amp"] > 0.5) & (dfc["amp"] < 10)  ]
df3 = dfc2.loc[:, ["id","market","name","previousClose","open","close","jump","amp","jumpRate","yVolume","預估量","量比","週量比","月量比","季量比","半年量比","年量比","周轉率","量比周轉"]].sort_values("半年量比", ascending=False)
display(df3)

ss = ''
o_nowTime = time.strftime("%m%d_%H%M", time.localtime())

for d in df3["id"].tolist():
  ss += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態2023{o_nowTime}.csv", ss)


目前時間：09:35,預估量放大係數:4.0


,id,market,name,previousClose,open,close,jump,amp,jumpRate,yVolume,預估量,量比,週量比,月量比,季量比,半年量比,年量比,周轉率,量比周轉
41,4976,Listed,佳凌,49.80,49.85,51.20,0.05,2.81,0.10,4095,47240,11.54,44.27,85.58,116.07,76.81,45.34,7.57,9.5536
3,3093,OTC,港建*,38.20,40.15,41.20,1.95,7.85,4.86,3481,28708,8.25,17.03,23.53,43.56,75.35,47.93,4.44,6.3454
11,5285,Listed,界霖,83.90,85.90,92.20,2.00,9.89,2.33,3728,33416,8.96,27.71,22.83,47.13,61.77,52.54,7.42,8.1893
73,6841,OTC,長佳智能,111.50,110.00,116.50,-1.50,4.48,-1.36,2885,4672,1.62,1.44,5.78,17.30,34.61,69.73,1.05,1.3328
35,3591,Listed,艾笛森,23.35,23.45,24.05,0.10,3.00,0.43,9256,18248,1.97,6.93,9.06,18.09,26.56,26.29,3.18,2.5748
44,3029,Listed,零壹,55.10,55.10,55.90,0.00,1.45,0.00,6132,18408,3.00,3.70,5.04,13.70,23.30,29.45,2.93,2.9666
36,3652,Listed,精聯,30.60,30.70,32.45,0.10,6.05,0.33,4947,7044,1.42,1.17,4.04,10.56,17.52,27.73,1.70,1.5589
12,2634,Listed,漢翔,39.40,40.30,41.25,0.90,4.70,2.23,12090,164932,13.64,13.97,7.73,9.90,14.83,14.11,4.19,8.9131
61,4164,Listed,承業醫,70.00,69.60,72.70,-0.40,3.86,-0.57,11929,17972,1.51,2.50,3.75,8.12,13.64,17.62,1.93,1.7182
18,4541,OTC,晟田,34.45,35.00,35.85,0.55,4.06,1.57,3761,16268,4.33,4.51,8.50,12.41,12.33,13.49,5.86,5.0971


In [ ]:
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
import _beowFmt as fm 

## 本益比、EPS
stockId = "2328"
pER = data.get('price_earning_ratio:本益比')
# print(pER[stockId])

EPS = data.get('fundamental_features:每股稅後淨利')
EPS成長率 = EPS.rolling(4).sum()/EPS.shift(4).rolling(4).sum() #最近四季的EPS/最近八季~四季的EPS總和推算
預估EPS = EPS*EPS成長率

print(EPS[stockId].tail(5))
print(EPS成長率[stockId].tail(5))

print(預估EPS[stockId].tail(5))

In [ ]:
## 自己擷取數量，只用來與昨天比較...
import _beowFmt as fm 
dd = dfb[["id","close","volume"]].sort_values("id")
ss = 'id,close,yVolume\n'
for d in dd.values.tolist():
  ss += f"{d[0]},{d[1]},{d[2]}\n"
fm.write_LogFile(f"volumeData/20230110.csv", ss)

# todo ##
# https://www.wantgoo.com/investrue/all-previous-weekly-candlestick?v=1672966860000  上一周收盤價


In [ ]:
import time
localtime = time.localtime()
nowTime = time.strftime("%H:%M", localtime)
print(nowTime)
# multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')

In [ ]:
import _beowFmt as fm 

df2 = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["volume"] >= 1000)].sort_values(by=['id'], ascending=True)
dd = df2[{"id","volume"}]
ss = ''
for d in dd.values.tolist():
  ss += f"{d[1]}, {d[0]}\n"

fm.write_LogFile(f"keepData/stockVolume.csv", ss)

In [ ]:
while True:
    localtime = time.localtime()
    result = time.strftime("%Y-%m-%d %I:%M:%S %p", localtime)
    print(result)
    time.sleep(1)

In [ ]:
from pathlib import Path  

filepath = Path('https://www.wantgoo.com/investrue/all-quote-info')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

In [ ]:
# importing the requests library
import requests

# api-endpoint
URL = "http://maps.googleapis.com/maps/api/geocode/json"

# location given here
location = "delhi technological university"

# defining a params dict for the parameters to be sent to the API
PARAMS = {'address':location}

# sending get request and saving the response as response object
r = requests.get(url = URL, params = PARAMS)

# extracting data in json format
data = r.json()


# extracting latitude, longitude and formatted address
# of the first matching location
latitude = data['results'][0]['geometry']['location']['lat']
longitude = data['results'][0]['geometry']['location']['lng']
formatted_address = data['results'][0]['formatted_address']

# printing the output
print("Latitude:%s\nLongitude:%s\nFormatted Address:%s"
	%(latitude, longitude,formatted_address))
